In [1]:
!pip uninstall torch torchvision fastai -y
!pip install \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch-1.0.0a0+1d94a2b-cp36-cp36m-linux_x86_64.whl  \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch_xla-0.1+5622d42-cp36-cp36m-linux_x86_64.whl
!pip install torchvision==0.2.0 

Uninstalling torch-1.4.0:
  Successfully uninstalled torch-1.4.0
Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Uninstalling fastai-1.0.60:
  Successfully uninstalled fastai-1.0.60
     |████████████████████████████████| 266.4MB 90.8MB/s 
     |████████████████████████████████| 57.9MB 82.9MB/s 
     |████████████████████████████████| 51kB 2.1MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torchvision import datasets
from skimage import io, transform
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision import transforms
import logging
from datetime import datetime

In [0]:
# Path of Folders
content = '/content/drive/My Drive/new-frames/'
labels_csv = '/content/drive/My Drive/weights.csv'
train_path = content + "train"
test_path = content + "test"
validation_path = content + "validation"


In [0]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

weights=pd.read_csv(labels_csv)
train_IDs, test_IDs = train_test_split(weights, test_size=0.30, random_state=42)

In [0]:
def generateDatasetIDS(list, root_folder = "/content/drive/My Drive/frames/"):
  """
  Return a list with all the image paths relative to those IDs
  """
  test_list = []
  for id in list:
    #print("Id:", id)
    for root, dirs, files in os.walk(root_folder + str(id) ):
        for filename in files:
            #print("\t", str(id) + "/" + filename)
            if('.png' in filename):
              add_file = str(id) + "/" + filename
              test_list.append(add_file)
  return test_list


class CustomListDataset(torch.utils.data.Dataset):

    def __init__(self, images_list, df_weights, root_dir,transform = None):
        """
        Args:
            images_list(list): list with IDs into the dataset
            root_dir(string): directory with all the images
            df_weights(pd_dataframe): dataframe with all the weights of the people
            transform: trasform operation for images
        """
        self.images_list = images_list
        self.root_dir = root_dir
        self.df_weights = df_weights
        self.df_weights = self.df_weights['WEIGHT'].values
        self.transform = transform

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images_list[idx])
        image = Image.open(img_name)
        #image = image.convert(mode='RGB')
        
        if(self.transform is not None):
            image = self.transform(image)

        frame_name = self.images_list[idx].split("/")
        id = int(frame_name[0])
        
        #labels = self.df_weights[id]
        labels = np.float(0)
        #print(labels)
        #sample = {'image': image, 'labels': labels}
        return image, torch.as_tensor(labels)

        #return sample


transform = transforms.Compose([
    # you can add other transformations in this list
    #transforms.Resize((148,148), interpolation=2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor()
    #,
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



In [0]:
'''class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # encoder
        self.enc1 = nn.Linear(in_features=150*150, out_features=512)
        self.enc2 = nn.Linear(in_features=512, out_features=256)
        self.enc3 = nn.Linear(in_features=256, out_features=64)
        self.enc4 = nn.Linear(in_features=64, out_features=32)
        self.enc5 = nn.Linear(in_features=32, out_features=64)

        # decoder 
        self.dec1 = nn.Linear(in_features=64, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=256)
        self.dec4 = nn.Linear(in_features=256, out_features=512)
        self.dec5 = nn.Linear(in_features=512, out_features=150*150)

    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        return x

net = Autoencoder()
print(net)'''

'class Autoencoder(nn.Module):\n    def __init__(self):\n        super(Autoencoder, self).__init__()\n\n        # encoder\n        self.enc1 = nn.Linear(in_features=150*150, out_features=512)\n        self.enc2 = nn.Linear(in_features=512, out_features=256)\n        self.enc3 = nn.Linear(in_features=256, out_features=64)\n        self.enc4 = nn.Linear(in_features=64, out_features=32)\n        self.enc5 = nn.Linear(in_features=32, out_features=64)\n\n        # decoder \n        self.dec1 = nn.Linear(in_features=64, out_features=32)\n        self.dec2 = nn.Linear(in_features=32, out_features=64)\n        self.dec3 = nn.Linear(in_features=64, out_features=256)\n        self.dec4 = nn.Linear(in_features=256, out_features=512)\n        self.dec5 = nn.Linear(in_features=512, out_features=150*150)\n\n    def forward(self, x):\n        x = F.relu(self.enc1(x))\n        x = F.relu(self.enc2(x))\n        x = F.relu(self.enc3(x))\n        x = F.relu(self.enc4(x))\n        x = F.relu(self.enc5(x))

In [9]:
import random

import torch
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from   torchvision import datasets, transforms


class AutoEncoder(nn.Module):
    
    def __init__(self, code_size):
        super().__init__()
        self.code_size = code_size
        
        # Encoder specification
        self.enc_cnn_1 = nn.Conv2d(1, 10, kernel_size=5)
        self.enc_cnn_2 = nn.Conv2d(10, 20, kernel_size=5)
        self.enc_linear_1 = nn.Linear(34 * 34 * 20, 150)
        #self.enc_linear_12 = nn.Linear(100, 150)
        self.enc_linear_2 = nn.Linear(150, self.code_size)
        
        # Decoder specification
        self.dec_linear_1 = nn.Linear(self.code_size, 150)
        self.dec_linear_2 = nn.Linear(150, IMAGE_SIZE)
        
    def forward(self, images):
        code = self.encode(images)
        out = self.decode(code)
        return out, code
    
    def encode(self, images):
        code = self.enc_cnn_1(images)
        code = F.selu(F.max_pool2d(code, 2))
        
        code = self.enc_cnn_2(code)
        code = F.selu(F.max_pool2d(code, 2))
        #print("first \t", code.shape)
        code = code.view([images.size(0), -1])
        #print("first \t", code.shape)
        code = F.selu(self.enc_linear_1(code))
        #code = F.selu(self.enc_linear_12(code))

        code = self.enc_linear_2(code)
        return code
    
    def decode(self, code):
        out = F.selu(self.dec_linear_1(code))
        #out = torch.sigmoid(self.dec_linear_12(out))
        out = torch.sigmoid(self.dec_linear_2(out))
        out = out.view([code.size(0), 1, IMAGE_WIDTH, IMAGE_HEIGHT])
        return out
    
IMAGE_SIZE = 150 * 150
IMAGE_WIDTH = IMAGE_HEIGHT = 150

# Hyperparameters
code_size = 64
num_epochs = 2000
batch_size = 64
lr = 0.003
optimizer_cls = optim.Adam

# Load data       train_IDs, test_IDs
train_list = generateDatasetIDS(train_IDs.ID,"/content/drive/My Drive/frames/")
train_custom = CustomListDataset(images_list = train_list, df_weights = weights, root_dir="/content/drive/My Drive/frames/", transform = transform)
train_loader = torch.utils.data.DataLoader(train_custom, batch_size=64, shuffle=False, num_workers=4)

test_list = generateDatasetIDS(test_IDs.ID,"/content/drive/My Drive/frames/")
test_custom = CustomListDataset(images_list = test_list, df_weights = weights, root_dir="/content/drive/My Drive/frames/", transform = transform)
test_loader = torch.utils.data.DataLoader(test_custom, batch_size=64, shuffle=False, num_workers=4)

# Instantiate model
autoencoder = AutoEncoder(code_size)
autoencoder = autoencoder.cuda()
loss_fn = nn.BCELoss()
optimizer = optimizer_cls(autoencoder.parameters(), lr=lr)

autoencoder.train()
# Training loop
for epoch in range(num_epochs):
    print("Epoch %d" % epoch)
    loss = 0

    for i, (images, _) in enumerate(train_loader):    # Ignore image labels
        #print(images.shape)
        img = Variable(images)
        img = img.cuda()

        out, code = autoencoder(img)
        
        optimizer.zero_grad()
        train_loss = loss_fn(out, img)
        #print(loss)
        train_loss.backward()
        optimizer.step()
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))



Epoch 0
epoch : 1/2000, loss = 0.213994
Epoch 1
epoch : 2/2000, loss = 0.082971
Epoch 2
epoch : 3/2000, loss = 0.082117
Epoch 3
epoch : 4/2000, loss = 0.082080
Epoch 4
epoch : 5/2000, loss = 0.082088
Epoch 5
epoch : 6/2000, loss = 0.082075
Epoch 6
epoch : 7/2000, loss = 0.082076
Epoch 7
epoch : 8/2000, loss = 0.082125
Epoch 8
epoch : 9/2000, loss = 0.082243
Epoch 9
epoch : 10/2000, loss = 0.082052
Epoch 10
epoch : 11/2000, loss = 0.082059
Epoch 11
epoch : 12/2000, loss = 0.082041
Epoch 12
epoch : 13/2000, loss = 0.082015
Epoch 13
epoch : 14/2000, loss = 0.081938
Epoch 14
epoch : 15/2000, loss = 0.081684
Epoch 15
epoch : 16/2000, loss = 0.081332
Epoch 16
epoch : 17/2000, loss = 0.081157
Epoch 17
epoch : 18/2000, loss = 0.081103
Epoch 18
epoch : 19/2000, loss = 0.081083
Epoch 19
epoch : 20/2000, loss = 0.081045
Epoch 20
epoch : 21/2000, loss = 0.081012
Epoch 21
epoch : 22/2000, loss = 0.080946
Epoch 22
epoch : 23/2000, loss = 0.080877
Epoch 23
epoch : 24/2000, loss = 0.080791
Epoch 24
ep

KeyboardInterrupt: ignored

In [0]:
loss = 0

for i, (images, _) in enumerate(test_loader):    # Ignore image labels
    #print(images.shape)
    img = Variable(images)
    img = img.cuda()

    out, code = autoencoder(img)
    
    
    test_loss = loss_fn(out, img)
    # add the mini-batch training loss to epoch loss
    loss += test_loss.item()

# compute the epoch training loss
loss = loss / len(test_loader)

# display the epoch training loss
print("Test Loss = {:.6f}".format(loss))

# Try reconstructing on test data
test_image, _ = x,y = test_custom.__getitem__(3)
test_image = Variable(test_image.view([1, 1, IMAGE_WIDTH, IMAGE_HEIGHT])).cuda()
test_reconst, _ = autoencoder(test_image)

torchvision.utils.save_image(test_image.data, 'orig.png')
torchvision.utils.save_image(test_reconst.data, 'reconst.png')

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f16e213be48>>
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f16e213be48>>
Traceback (most recent call last):
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f16e213be48>>
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f16e213be48>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717,

In [0]:
torch.save(autoencoder.state_dict(), "/content/drive/My Drive/model/autoencoder16Transformed_features.pt") 
